## Non Compliance Experiment=2
Test top norms for different w_nc, and relation between violations and 

In [1]:
import sys
sys.path.append('../src')

In [2]:
import yaml
from IPython.utils import io
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
from mcmc_norm_learning.algorithm_1_v4 import to_tuple

In [3]:
def write_log(output,path):
    with open(Path(path), 'w') as f:
        f.write(output)

### Loop over w_nc values

In [4]:
# !rm -r ../data_nc/exp_nc2/*

In [5]:
w_nc_list_part1=[0.0,0.05,0.1,0.2]

In [6]:
#Stopped after w_nc=0.1, due to hight compute time
for w_nc in tqdm(w_nc_list_part1,"Loop Over diff w_nc"):
    exp_dir_name="exp_nc2/w_nc={}".format(w_nc)
    with io.capture_output() as exp_log:
        !python ../scripts/nc_experiments.py -exp $exp_dir_name -w_nc $w_nc  -n_threads 6
    write_log(exp_log.stdout,"../data_nc/{}/run.log".format(exp_dir_name)) 

In [7]:
w_nc_list_part2=[0.15,0.2]

In [8]:
#Stopped after w_nc=0.1, due to hight compute time
for w_nc in tqdm(w_nc_list_part2,"Loop Over diff w_nc"):
    exp_dir_name="exp_nc2/w_nc={}".format(w_nc)
    with io.capture_output() as exp_log:
        !python ../scripts/nc_experiments.py -exp $exp_dir_name -w_nc $w_nc  -n_threads 6
    write_log(exp_log.stdout,"../data_nc/{}/run.log".format(exp_dir_name)) 

In [ ]:
# w_nc_list_part3=[0.25,0.3,0.35,0.4,0.45]

## Analysis
### 1. True Norm log_posterior for diff w_nc

In [14]:
w_nc_list = [0.0,0.05,0.1,0.15,0.2]

In [15]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc2/w_nc={}".format(w_nc)
    prior_out=!grep "For True Norm" ../data_nc/{exp_dir_name}/run.log
    post_out=!grep "log_post_true_norm" ../data_nc/{exp_dir_name}/run.log
    print (f"For w_nc={w_nc}:")
    print (prior_out)
    print (post_out)
    print()

For w_nc=0.0:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
[]

For w_nc=0.05:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
[]

For w_nc=0.1:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
[]

For w_nc=0.15:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
[]

For w_nc=0.2:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
[]



### 2. True Norm atleast mentioned in any chains, for diff w_nc

In [16]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc2/w_nc={}".format(w_nc)
    out=!grep "True norm in some chain(s)" ../data_nc/{exp_dir_name}/chain_info.txt
    print (f"For w_nc={w_nc}, True Norm found={out}")

For w_nc=0.0, True Norm found=['True norm in some chain(s): True']
For w_nc=0.05, True Norm found=['True norm in some chain(s): True']
For w_nc=0.1, True Norm found=['True norm in some chain(s): False']
For w_nc=0.15, True Norm found=['True norm in some chain(s): True']
For w_nc=0.2, True Norm found=['True norm in some chain(s): True']


### 3. Rank of True Norm as per log_post, iff it was found

In [17]:
with open("../params_nc.yaml", 'r') as fd:
    params = yaml.safe_load(fd)
true_norm=to_tuple(params['true_norm']['exp'])

In [19]:
for w_nc in w_nc_list:
    exp_dir_name="exp_nc2/w_nc={}".format(w_nc)
    rank_df=pd.read_csv(f"../data_nc/{exp_dir_name}/ranked_posteriors.csv",index_col=False)
    rank_post=rank_df.loc[rank_df.expression==str(true_norm)][["post_rank","log_posterior"]].values
    if rank_post.shape[0]==1:
        print (f"For w_nc={w_nc}, True Norm rank={rank_post[0][0]},wi posterior={rank_post[0][1]}")

For w_nc=0.0, True Norm rank=1.0,wi posterior=-25176.704950912368
For w_nc=0.05, True Norm rank=1.0,wi posterior=-25781.819948105094
For w_nc=0.15, True Norm rank=1.0,wi posterior=-25772.77641199651
For w_nc=0.2, True Norm rank=1.0,wi posterior=-25883.433244964857


#### ==> True Norm always has highest posterior, when its found,
confirm if it is in one chain, or multiple

### 4. In how many chains, max post crosses post(true_norm)

In [20]:
stacked=pd.DataFrame()
for w_nc in w_nc_list:
    exp_dir_name="exp_nc2/w_nc={}".format(w_nc)
    result = pd.read_csv(f"../data_nc/{exp_dir_name}/chain_posteriors_nc.csv")
    result["w_nc"] = w_nc
    stacked=stacked.append(result)

In [21]:
grouped=stacked.groupby(["w_nc","chain_number"])[["log_posterior"]].agg(['min', 'max', 'mean', 'std'])

In [29]:
true_post={
    "a":-25177,"b":-25782,\
    "c":1,"d":-25773,\
    "e":-25884,"f":1
}

In [30]:
grouped["w_nc"]=[x[0] for x in grouped.index]
grouped["key"]=np.where(grouped["w_nc"]==0.0,"a",\
                        np.where(grouped["w_nc"]==0.05,"b",\
                                np.where(grouped["w_nc"]==0.1,"c",\
                                        np.where(grouped["w_nc"]==0.15,"d",\
                                                np.where(grouped["w_nc"]==0.2,"e","f")\
                                                )\
                                        )\
                                )\
                        )

In [31]:
grouped.style.apply(lambda x: ['background: lightgreen' if x['log_posterior',  'max'] >=\
                               true_post[x["key"].values.item()] else '' for i in x], 
               axis=1)

### 5. If True Norm, features in Top Norms

In [32]:
top_norms=stacked.groupby(["w_nc","expression"]\
               )[["chain_number","log_posterior"\
                 ]].agg({"log_posterior":'mean',"chain_number":"count"}\
                       ).sort_values(by=["chain_number"]\
                                    ).groupby(["w_nc"\
                                              ]).tail(20).sort_values(by=["w_nc","chain_number"],\
                                                                     ascending=[True, False])

In [33]:
top_norms["grp_rnk"]=top_norms.groupby("w_nc")[["chain_number"]].rank(method='dense',ascending=False)

In [34]:
top_norms.loc[0.0].head(10).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["a"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25176.704951,4745,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25176.704951,4285,2.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'b'], ['Shape', 'any']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'triangle'], ['PerZone', '2']]]",-26772.561281,2225,3.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'triangle'], ['Zone', '3'], ['Next-Move', ['Colour', 'g'], ['Shape', 'any']]], ['Zone', '1']]]",-26542.237328,1951,4.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25177.803563,1604,5.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'b'], ['Shape', 'any']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'triangle'], ['PerZone', '2']]]",-26773.659894,1029,6.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '3'], ['Next-Move', ['Colour', 'b'], ['Shape', 'any']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'triangle'], ['PerZone', '2']]]",-26773.659894,898,7.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '3'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'b'], ['Shape', 'any']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'b'], ['Shape', 'triangle'], ['PerZone', '2']]]",-26773.659894,842,8.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'b'], ['Shape', 'square']]], ['Zone', '1']]]",-26774.044521,729,9.000000


In [35]:
top_norms.loc[0.05].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["b"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'square']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'g'], ['Shape', 'any'], ['PerZone', '2']]]",-26655.511829,4274,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25781.819948,2973,2.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25781.819948,2561,3.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '3'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Next-Move', ['Colour', 'any'], ['Shape', 'square']]]], ['Zone', '3']]]",-26706.745893,2510,4.000000


In [39]:
true_norm

('Norms',
 ('Obl',
  ('Moved',
   ('Colour', 'r'),
   ('Shape', 'any'),
   ('Zone', '1'),
   ('Next-Move', ('Colour', 'any'), ('Shape', 'any'))),
  ('Zone', '2')),
 ('Per',
  ('Action', 'putdown'),
  ('Colour', 'any'),
  ('Shape', 'square'),
  ('PerZone', '3')))

#### Below top norms(1,2) equivalent norm to true norm for a very peculiar action seq:
1. posterior is lowered due to low prior (~complex condition)

In [36]:
top_norms.loc[0.1].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["c"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norm', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '2'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]]], ['Zone', '2']]]",-26716.599977,3941,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'b'], ['Shape', 'square'], ['PerZone', '3']]]",-26392.178876,2594,2.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'b'], ['Shape', 'square'], ['PerZone', '3']]]",-26392.178876,2561,3.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'triangle']]], ['Zone', '2']]]",-26206.018037,1999,4.000000


In [37]:
top_norms.loc[0.15].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["d"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25772.776412,3554,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25772.776412,3334,2.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '2'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]]], ['Zone', '1']]]",-26723.138570,2906,3.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25773.875024,1358,4.000000


In [38]:
top_norms.loc[0.2].head(10).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
                               true_post["e"] else '' for i in x], 
               axis=1)

,log_posterior,chain_number,grp_rnk
expression,,,
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25883.433245,5239,1.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25883.433245,3505,2.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'any'], ['PerZone', '1']]]",-26767.543194,2988,3.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']]]",-25993.578671,1960,4.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'any'], ['Zone', '1'], ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '3'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'any'], ['PerZone', '1']]]",-26767.543194,1899,5.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'any'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']]]",-25993.578671,1891,6.000000
"['Norm', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '3'], ['Next-Move', ['Colour', 'b'], ['Shape', 'any']]], ['Zone', '2']]]",-26750.758130,1433,7.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'r'], ['Shape', 'circle'], ['Zone', '1'], ['Next-Move', ['Colour', 'any'], ['Shape', 'any']]], ['Zone', '2']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'square'], ['PerZone', '3']]]",-25884.531857,1347,8.000000
"['Norms', ['Obl', ['Moved', ['Colour', 'b'], ['Shape', 'any'], ['Zone', '1'], ['Next-Move', ['Colour', 'r'], ['Shape', 'any']]], ['Zone', '3']], ['Per', ['Action', 'putdown'], ['Colour', 'any'], ['Shape', 'any'], ['PerZone', '1']]]",-26623.739948,1263,9.000000
